# Cars 4 You: Expediting Car Evaluations with Machine Learning

## Problem Statement

**Cars 4 You** is an online car resale company that buys vehicles from different brands through its platform.  
Users submit details about their cars, which are then sent for mechanic evaluation before resale.

Due to rapid growth, the company faces **delays in car inspections**, leading to loss of potential customers.  
To solve this, the company wants a **predictive model** capable of estimating the **price of a car based on user-provided inputs**, without needing a physical evaluation.

## Project Objectives

The overall project is structured into three main goals (for the final submission), but the **first deliverable** focuses on the **initial stages** of this workflow.

### 1. Regression Benchmarking
Develop a regression model that predicts car prices (`price`) using the 2020 Cars 4 You dataset.  
Implement a consistent **model assessment strategy** to compare models and select the most generalizable one.

### 2. Model Optimization
Later in the project, explore model optimization through:
- Hyperparameter tuning  
- Feature engineering and preprocessing adjustments  

Compare and discuss the performance improvements.

### 3. Additional Insights
Open-ended exploration for the final project phase (e.g., feature importance, ablation study, specialized models, deep learning).



## Dataset Description

You have access to two datasets:

- **Training Set:** Cars from the 2020 Cars 4 You database, including prices (`price`).
- **Test Set:** Same descriptive features but without prices — used for Kaggle submission.

### Variables

| Attribute | Description |
|------------|-------------|
| **carID** | Unique identifier for each car |
| **Brand** | Car brand (e.g., Ford, Toyota) |
| **model** | Car model |
| **year** | Year of registration |
| **mileage** | Distance travelled (in miles) |
| **tax** | Road tax applicable in 2020 (in £) |
| **fuelType** | Fuel type (Diesel, Petrol, Hybrid, Electric) |
| **mpg** | Miles per gallon |
| **engineSize** | Engine size in liters |
| **paintQuality%** | Mechanic’s paint and hull assessment |
| **previousOwners** | Number of previous owners |
| **hasDamage** | Boolean (True/False) — damaged or not |
| **price** | Target variable — purchase price (£) |



## FIRST DELIVERABLE — Task Overview

This initial assignment is a **preparatory milestone** for your final project.  
You will **explore, prepare, and model** the dataset to demonstrate an understanding of the full ML pipeline.

Your submission must include:

1. A **Jupyter Notebook** (or a ZIP of notebooks) with code and explanatory Markdown cells.
2. A **2-page PDF** describing your pipeline’s structure and rationale.

### Naming Convention
Homework_Group36 



## DELIVERABLE TASKS & POINT DISTRIBUTION

### 1. Data Import & Exploration — *3 points*
- Load and inspect the dataset contents.  
- Generate **descriptive statistics**.  
- Detect and describe **inconsistencies or anomalies**.  
- Perform **visual exploration** (univariate & multivariate).  
- Extract **insights** and explain your findings.


### 2. Data Cleaning & Preprocessing — *5 points*
- Identify and **handle missing values** appropriately.  
- Detect and manage **outliers**, justifying your choices.  
- Encode **categorical variables**.  
- Evaluate existing features and, if relevant, **create new ones** with explanations.  
- Apply **data scaling/normalization**, explaining why your approach fits the problem.


### 3. Feature Selection — *3 points*
- Define a **clear feature selection strategy**.  
- Use **course-approved methods** (e.g., correlation, recursive elimination, model-based).  
- Present and justify your **final feature set**.


### 4. Model Building & Performance Assessment — *4 points*
- Identify the **problem type** (regression).  
- Choose relevant **algorithms** (linear models, tree-based, etc.).  
- Define one **model assessment strategy** (e.g., holdout, k-fold cross-validation).  
- Justify the **evaluation metrics** (e.g., RMSE, MAE, R²).  
- Train **at least one model** and generate **predictions for the test set**.

💡 **Bonus:** +1 point if your team ranks in the **Top 5 on Kaggle**.



### 5. Pipeline Structure & Documentation (PDF) — *5 points*
- Provide a **schematic of your full pipeline**:
  - Data cleaning  
  - Feature preprocessing  
  - Feature selection  
  - Model training and evaluation  
- Include concise explanations of:
  - What is done  
  - To which variables  
  - Why each step is taken
- Maintain **clarity, structure, and readability** throughout.


### Total Scoring
| Component | Points |
|------------|---------|
| Data Exploration | 3 |
| Preprocessing | 5 |
| Feature Selection | 3 |
| Model Building | 4 |
| Pipeline Documentation | 5 |
| **Total** | **20 points** |
| **Bonus (Top 5 Kaggle)** | **+1** |
| **Final Grade = min(20, your points)** |  |


## Completion Checklist

| Task | Done? |
|------|-------|
| Dataset imported and explored | ☐ |
| Missing values handled | ☐ |
| Outliers addressed | ☐ |
| Categorical variables encoded | ☐ |
| Features scaled and/or engineered | ☐ |
| Feature selection strategy implemented | ☐ |
| Regression model trained | ☐ |
| Performance metrics explained | ☐ |
| Predictions generated for test set | ☐ |
| Pipeline described in PDF | ☐ |
| Kaggle submission tested | ☐ |
| Notebook clean, commented, and executable | ☐ |


# 1. Importing libraries & Data

In [1]:
# general imports that we will need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# data partition
from sklearn.model_selection import train_test_split

# filter methods
# - spearman 
# - chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# embedded methods
from sklearn.linear_model import LassoCV

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Custom functions and variables
from functions import histogram_boxplot, labeled_barplot, stacked_barplot, distribution_plot_wrt_target 
from vocab_prep import BRAND_MODEL_VOCAB

#set random seed for reproducibility
RSEED = 42
np.random.seed(RSEED)
pd.set_option('display.max_rows', 500)

#### Data Loading

In [2]:
# this loads the train data 
train_raw = pd.read_csv('project_data/train.csv')

# this loads the test data
test_raw = pd.read_csv('project_data/test.csv')

#print first five rows of the training data
train_raw.head()

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
0,69512,VW,Golf,2016.0,22290,Semi-Auto,28421.0,Petrol,NaN,11.417268,2.0,63.0,4.000000,0.0
1,53000,Toyota,Yaris,2019.0,13790,Manual,4589.0,Petrol,145.0,47.900000,1.5,50.0,1.000000,0.0
2,6366,Audi,Q2,2019.0,24990,Semi-Auto,3624.0,Petrol,145.0,40.900000,1.5,56.0,4.000000,0.0
3,29021,Ford,FIESTA,2018.0,12500,anual,9102.0,Petrol,145.0,65.700000,1.0,50.0,-2.340306,0.0
4,10062,BMW,2 Series,2019.0,22995,Manual,1000.0,Petrol,145.0,42.800000,1.5,97.0,3.000000,0.0


#### Verify that carID can be used as index

In [3]:
train_raw['carID'].is_unique

True

In [4]:
train_raw['carID'].isna().any()

False

#### Set carID as index

In [5]:
# setting carID as index
train_raw.set_index('carID', inplace = True)

# sorting by index
train_raw.sort_index(inplace =True)

# displaying the first three rows
train_raw.head(3)

,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
carID,,,,,,,,,,,,,
0,Aud,A3,2016.0,10990,Manual,57097.0,Diesel,0.0,83.1,1.6,75.0,4.0,0.0
1,Audi,A7,2017.0,22999,Semi-Auto,24337.0,diesel,150.0,55.4,3.0,31.0,0.0,0.0
2,Audi,A4,2016.0,14242,Manual,14900.0,Petrol,125.0,53.3,1.4,65.0,4.0,0.0


# 2. Exploratory Data Analysis

In [6]:
train = train_raw.copy()

In [7]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Brand,74452,72,Ford,14808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model,74456,735,Focus,6353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,74482.0,NaN,NaN,NaN,2017.096611,2.208704,1970.0,2016.0,2017.0,2019.0,2024.121759
price,75973.0,NaN,NaN,NaN,16881.889553,9736.926322,450.0,10200.0,14699.0,20950.0,159999.0
transmission,74451,40,Manual,38050,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mileage,74510.0,NaN,NaN,NaN,23004.184088,22129.788366,-58540.574478,7423.25,17300.0,32427.5,323000.0
fuelType,74462,34,Petrol,37995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tax,68069.0,NaN,NaN,NaN,120.329078,65.521176,-91.12163,125.0,145.0,145.0,580.0
mpg,68047.0,NaN,NaN,NaN,55.152666,16.497837,-43.421768,46.3,54.3,62.8,470.8
engineSize,74457.0,NaN,NaN,NaN,1.660136,0.573462,-0.103493,1.2,1.6,2.0,6.6


In [8]:
# let's check the data types
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75973 entries, 0 to 75972
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           74452 non-null  object 
 1   model           74456 non-null  object 
 2   year            74482 non-null  float64
 3   price           75973 non-null  int64  
 4   transmission    74451 non-null  object 
 5   mileage         74510 non-null  float64
 6   fuelType        74462 non-null  object 
 7   tax             68069 non-null  float64
 8   mpg             68047 non-null  float64
 9   engineSize      74457 non-null  float64
 10  paintQuality%   74449 non-null  float64
 11  previousOwners  74423 non-null  float64
 12  hasDamage       74425 non-null  float64
dtypes: float64(8), int64(1), object(4)
memory usage: 8.1+ MB


In [9]:
# let's check a sample of the data
train.sample(10)

,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
carID,,,,,,,,,,,,,
69512,VW,Golf,2016.0,22290,Semi-Auto,28421.000000,Petrol,NaN,11.417268,2.0,63.0,4.000000,0.0
53000,Toyota,Yaris,2019.0,13790,Manual,4589.000000,Petrol,145.0,47.900000,1.5,50.0,1.000000,0.0
6366,Audi,Q2,2019.0,24990,Semi-Auto,3624.000000,Petrol,145.0,40.900000,1.5,56.0,4.000000,0.0
29021,Ford,FIESTA,2018.0,12500,anual,9102.000000,Petrol,145.0,65.700000,1.0,50.0,-2.340306,0.0
10062,BMW,2 Series,2019.0,22995,Manual,1000.000000,Petrol,145.0,42.800000,1.5,97.0,3.000000,0.0
14704,BMW,3 Series,2014.0,8199,anual,101153.348285,Diesel,30.0,65.700000,2.0,74.0,0.000000,0.0
6924,Audi,A3,2017.0,16089,Manual,21396.000000,Petrol,20.0,60.100000,1.4,75.0,4.000000,0.0
50783,Skoda,Octavia,2017.0,7500,Manual,116750.000000,Diesel,145.0,68.900000,1.6,59.0,4.000000,0.0
67071,VW,Passat,2017.0,15999,Semi-Auto,30339.000000,Diesel,150.0,62.800000,2.0,85.0,4.000000,0.0


## 2.1 Inconsistencies

**Brand**

Typing errors

In [11]:
# display all unique values 
train['Brand'].unique()

array(['Aud', 'Audi', nan, 'audi', 'udi', 'AUDI', 'UDI', 'ud', 'aud',
       'AUD', 'BMW', 'MW', 'bmw', 'BM', 'mw', 'Ford', 'ford', 'for',
       'FORD', 'For', 'ord', 'ORD', 'FOR', 'or', 'Hyundai', 'Hyunda',
       'yundai', 'hyundai', 'HYUNDAI', 'hyunda', 'HYUNDA', 'yunda',
       'Mercedes', 'mercedes', 'MERCEDES', 'ercedes', 'Mercede', 'ercede',
       'ERCEDES', 'MERCEDE', 'mercede', 'Skoda', 'koda', 'SKODA', 'Skod',
       'SKOD', 'skoda', 'kod', 'skod', 'KODA', 'Toyota', 'toyota',
       'TOYOTA', 'Toyot', 'oyota', 'TOYOT', 'OYOTA', 'toyot', 'Opel',
       'pel', 'opel', 'Ope', 'OPEL', 'OPE', 'PEL', 'ope', 'pe', 'VW', 'W',
       'v', 'V', 'vw', 'w'], dtype=object)

In [12]:
# change all values to lower case and strip starting and ending spaces
train['Brand'] = train['Brand'].str.lower().str.strip()

np.array(sorted(train['Brand'].dropna().unique()))

array(['aud', 'audi', 'bm', 'bmw', 'ercede', 'ercedes', 'for', 'ford',
       'hyunda', 'hyundai', 'kod', 'koda', 'mercede', 'mercedes', 'mw',
       'ope', 'opel', 'or', 'ord', 'oyota', 'pe', 'pel', 'skod', 'skoda',
       'toyot', 'toyota', 'ud', 'udi', 'v', 'vw', 'w', 'yunda', 'yundai'],
      dtype='<U8')

**model**

Typing errors

In [13]:
print(f'The number of unique model values is {train['model'].unique().size}')

The number of unique model values is 736


In [14]:
# change all values to lower case and strip starting and ending spaces
train['model'] = train['model'].str.lower().str.strip()

print(f'After lowercase and strip the number of unique values is {train['model'].unique().size}')

After lowercase and strip the number of unique values is 297


In [15]:
train['model'].unique()

array(['a3', 'a7', 'a4', 'q3', 'a6', 'a5', 'tt', 'q7', 'a1', 'q2', 'rs5',
       'a8', 'q5', 'a', 's4', nan, 'q', 'q8', 'rs3', 'rs6', 'rs4', 'rs',
       's3', 'r8', 's8', 'sq5', 'a2', 'sq7', 's5', 't', '1 series',
       '3 series', '2 series', 'x1', 'x3', 'x5', '5 series', 'z4',
       '4 series', 'x6', 'x2', '2 serie', '5 serie', '6 series', 'x4',
       '7 series', '3 serie', 'm3', 'm4', 'i8', 'x', 'x7', 'm2', 'm',
       'm5', '4 serie', '1 serie', 'z3', 'i3', 'z', '8 series', 'm6',
       '7 serie', '8 serie', 'i', '6 serie', 'focus', 'focu', 'fiesta',
       'kuga', 'puma', 'ecosport', 'ka+', 'edge', 's-max', 'fusion',
       'fiest', 'c-max', 'ka', 'grand c-max', 'mondeo', 's-ma', 'monde',
       'tourneo connect', 'b-max', 'galaxy', 'mustang',
       'grand tourneo connect', 'tourneo custom', 'c-ma', 'kug', 'b-ma',
       'ecospor', 'grand c-ma', 'streetka', 'k', 'ranger', 'escort',
       'galax', 'edg', 'grand tourneo connec', 'tourneo custo', 'i30',
       'ix20', 'i10', 'i

**transmission**

Typing errors

In [16]:
train['transmission'].unique()

array(['Manual', 'Semi-Auto', 'Automatic', 'anual', 'manual', 'AUTOMATIC',
       'Manua', 'emi-Auto', nan, 'unknown', 'SEMI-AUTO', 'Semi-Aut',
       'nknown', 'Manual ', 'MANUAL', 'automatic', ' Manual ',
       'semi-auto', 'emi-auto', 'Automati', 'utomatic', 'anua', 'ANUAL',
       'unknow', 'SEMI-AUT', 'UTOMATIC', 'emi-Aut', 'MANUA', 'semi-aut',
       'EMI-AUTO', 'utomati', 'UNKNOWN', ' Manual', ' MANUAL ', 'manua',
       'AUTOMATI', ' manual ', 'manual ', 'Other', 'automati', 'nknow'],
      dtype=object)

In [17]:
# change all values to lower case and strip starting and ending spaces
train['transmission'] = train['transmission'].str.lower().str.strip()

np.array(sorted(train['transmission'].dropna().unique()))

array(['anua', 'anual', 'automati', 'automatic', 'emi-aut', 'emi-auto',
       'manua', 'manual', 'nknow', 'nknown', 'other', 'semi-aut',
       'semi-auto', 'unknow', 'unknown', 'utomati', 'utomatic'],
      dtype='<U9')

**fuelType**

Typing errors

In [18]:
train['fuelType'].unique()

array(['Diesel', 'diesel', 'Petrol', 'Petro', 'etrol', 'petrol', 'iesel',
       'PETROL', nan, 'DIESEL', 'Diese', 'Hybrid', 'etro', 'ETROL',
       'iese', 'PETRO', 'petro', 'ybrid', 'diese', 'IESEL', 'Other',
       'HYBRID', 'hybrid', 'Hybri', 'DIESE', 'Othe', 'Electric', 'ETRO',
       'IESE', 'ybri', 'YBRID', 'OTHER', 'ther', 'HYBRI', 'other'],
      dtype=object)

In [19]:
# change all values to lower case and strip starting and ending spaces
train['fuelType'] = train['fuelType'].str.lower().str.strip()

np.array(sorted(train['fuelType'].dropna().unique()))

array(['diese', 'diesel', 'electric', 'etro', 'etrol', 'hybri', 'hybrid',
       'iese', 'iesel', 'othe', 'other', 'petro', 'petrol', 'ther',
       'ybri', 'ybrid'], dtype='<U8')

**year**

Non int values

In [20]:
print(train['year'].unique())
print("")
print(f'The number of rows with non-integer year values are: {train.loc[train['year'] != round(train['year']), ['year']].size}')

[2016.         2017.         2019.         2015.         2020.
 2018.         2014.         2013.         2011.         2012.
 2023.60527574 2008.         2005.                   nan 2010.37154646
 2002.         2007.         2006.         2010.         2004.
 2003.         2009.         1998.         2024.12175905 2009.81675711
 2001.         1997.         1999.         2023.1169636  2010.67696784
 2000.         1996.         2022.87800554 2011.11118842 2023.97731126
 2011.21085349 1970.         2022.69668507 2012.69574039 2023.36707842
 2010.26863473 2010.56500919 2023.26798867 2023.38982198 2010.7464032 ]

The number of rows with non-integer year values are: 2214


**mileage**

Negative values

In [21]:
print(f'The number of rows with negative mileage values is: {train.loc[train['mileage'] < (0), ['mileage']].size}')

The number of rows with negative mileage values is: 369


**tax**

Negative values

In [22]:
print(train['tax'].unique())
print("")
print(f'The number of rows with negative tax values is: {train.loc[train['tax'] < (0), ['tax']].size}')

[  0.         150.         125.         145.                  nan
  30.          20.         235.         555.         200.
 -60.6840948  160.         205.         240.         260.
 265.         330.         327.91577835 300.         295.
 565.         325.         195.         135.         165.
 305.         535.         140.         570.         280.
 315.         540.         220.         580.         190.
 155.         230.         -43.24208237 316.48718937 270.
 120.         255.         250.         110.         185.
 245.         -45.78061049 299.47350391 290.         115.
 210.         294.84340933 -31.91494466 -56.19320154 326.02823317
 520.          10.         277.38425448 -18.33074314 -91.12163048
 316.29630636 290.48678508 -20.24107949 130.         -49.15354066
 303.14889146]

The number of rows with negative tax values is: 378


**mpg**

Negative values

In [23]:
print(train['mpg'].unique())
print("")
print(f'The number of rows with negative mpg values is: {train.loc[train['mpg'] < (0), ['mpg']].size}')

[ 83.1         55.4         53.3         51.4         47.9
  57.6         58.9         46.3         49.6         33.2
  40.9         32.1         31.4         47.1         54.3
  80.7         64.2         39.8         68.9         74.3
  44.1         38.7         38.2         67.3         44.8
  26.4         39.2         42.8         42.2         48.7
  60.1         56.5         32.8         50.4         52.3
  41.5         43.5         32.5         61.4         40.4
  65.7                 nan  34.9         37.2         70.6
  45.6         35.8          9.82146197  34.          34.5
  22.4         76.3         62.8         33.6         30.7
  89.77369267  36.7         31.7         29.4         30.1
  36.2         37.7         29.7         29.1         72.4
  35.3        141.3        117.7         31.          34.4
  57.7        156.9         21.1         28.8        176.6
  42.1         33.3         20.3        188.3         76.4
  19.6         21.9         24.1         21.6         22

**engineSize**	

Negative values

In [24]:
print(train['engineSize'].unique())
print("")
print(f'The number of rows with negative engineSize values is: {train.loc[train['engineSize'] < (0), ['engineSize']].size}')

[ 1.6         3.          1.4         2.          1.8         1.
  1.5                nan  4.2         1.2         0.15452932  0.
  2.5         4.          3.73923548  2.9         2.7         5.2
  1.9         4.1         3.2         0.31340658  3.82275785  2.2
  4.4         6.6         0.6         3.5         5.          1.1
 -0.1034928   2.3         2.63120047  1.3         1.7         2.6631132
  0.37112737  2.4         2.1         3.72371217  0.30707394  5.5
  5.4         6.2         3.6         4.7         2.6         6.
  4.3         0.19146242  2.61964142  0.16286595  2.8         2.78177621
  4.5         0.7395339   2.06411863  0.18642488  2.98579344]

The number of rows with negative engineSize values is: 84


**paintQuality%**

% above 100 & non int values 

In [25]:
print(train['paintQuality%'].unique())
print("")
print(f'The number of rows with paintQuality% values above 100% is: {train.loc[train['paintQuality%'] > (100), ['paintQuality%']].size}')
print(f'The number of rows with non-integer paintQuality% values are: {train.loc[train['paintQuality%'] != round(train['paintQuality%']), ['paintQuality%']].size}')

[ 75.          31.          65.          55.          62.
  97.          77.          49.          79.          96.
  36.         125.30194543  80.          57.          83.
  47.          66.          30.          74.          85.
  99.          73.          67.          44.          45.
  56.          52.          81.           3.1152953   54.
          nan  78.          72.          82.          39.
  76.          34.          59.          48.          88.
  53.          41.          35.          94.          95.
  87.          70.          37.          51.          58.
  61.          86.          98.          60.          32.
  92.          84.          71.          91.          38.
  90.          50.          40.          33.          89.
  69.          42.          68.          43.          93.
  63.          46.          64.         125.5694986    3.14037046
   3.22574362 125.10995148   3.25476013 125.18872876 125.20203342
   3.20741784   2.72515272 125.4535988    1.63891309 125

**previousOwners**	

Negative & non int values

In [26]:
print(train['previousOwners'].unique())
print("")
print(f'The number of rows with negative previousOwners values is: {train.loc[train['previousOwners'] < (0), ['previousOwners']].size}')
print(f'The number of rows with non-integer previousOwners values are: {train.loc[train['previousOwners'] != round(train['previousOwners']), ['previousOwners']].size}')

[ 4.          0.          1.          2.          3.                 nan
  6.2482512  -2.33512284 -2.31225953  6.23017958  6.25823052 -2.34030622
  6.22789796 -2.31733109  6.25837066 -2.33936045  6.24177863 -2.33744529
  6.21772443 -2.34010209 -2.29943868  6.23308217 -2.34565     6.24583495]

The number of rows with negative previousOwners values is: 371
The number of rows with non-integer previousOwners values are: 2284


**hasDamage**

Only zeros

In [27]:
print(train['hasDamage'].unique())

[ 0. nan]


## 2.1.1 Adress Inconsistencies

**Vocabulary**

In [44]:
# Correct misspelled values in 'Brand' column
BRAND_CORRECTIONS = {
    'aud': 'audi',
    'udi': 'audi',
    'ud': 'audi',
    'mw': 'bmw',
    'bm': 'bmw',
    'for': 'ford',
    'ord': 'ford', 
    'or': 'ford',
    'hyunda': 'hyundai',
    'yundai': 'hyundai',
    'yunda': 'hyundai',
    'mercedes': 'mercedes-benz',
    'mercede': 'mercedes-benz',
    'ercedes': 'mercedes-benz',
    'ercede': 'mercedes-benz',
    'mercedes benz': 'mercedes-benz',
    'koda': 'skoda',
    'skod': 'skoda',
    'kod': 'skoda',
    'toyot': 'toyota',
    'oyota': 'toyota',
    'pel': 'opel',
    'pe': 'opel',
    'ope': 'opel',
    'vw': 'volkswagen',
    'v': 'volkswagen',
    'w': 'volkswagen'
}

Transmission_Corrections = {
    'anual': 'manual',
    'manua': 'manual',
    'anua': 'manual',
    'emi-auto': 'semi-auto',
    'semi-aut': 'semi-auto',
    'emi-aut': 'semi-auto',
    'automati': 'automatic',
    'utomatic': 'automatic',
    'utomati': 'automatic',
    'nknown': 'unknown',
    'unknow': 'unknown',
    'other': 'unknown',
    'nknow': 'unknown'
}

fuelType_Corrections = {
    'etro': 'petrol',
    'etrol': 'petrol',
    'petro': 'petrol',
    'hybri': 'hybrid',
    'ybri': 'hybrid',
    'ybrid': 'hybrid',
    'iese': 'diesel',
    'iesel': 'diesel',
    'othe': 'other',
    'ther': 'other'
}


#### model vocabulary

In [29]:
pure_brand_model_df = pd.DataFrame(BRAND_MODEL_VOCAB)
#pure_brand_model_df

In [30]:
# Explain why the count of unique models (both correct and typos) 
# differs between the 'vocabulary' (pure_brand_model_df) and the train dataset.
set1 = set(pure_brand_model_df['model_lower'].dropna().unique())
set2 = set(train['model'].dropna().unique())
print("len(set1)", len(set1))
print("len(set2)", len(set2))  
print("len(set2)-len(set1)", len(set2)-len(set1))
print("set(train['model']) - set(pure_brand_model_df['model_lower'])", set2 - set1)
print("set(pure_brand_model_df['model_lower']) - set(train['model'])", set1 - set2)

len(set1) 288
len(set2) 296
len(set2)-len(set1) 8
set(train['model']) - set(pure_brand_model_df['model_lower']) {'viv', 'q', 'kadjar', 'rs', 'm', 'a', 'z', 'x', 'i', 't'}
set(pure_brand_model_df['model_lower']) - set(train['model']) {'suvra', 'veloster'}


In [31]:
# Fix model names only for rows with a known (non-NaN) Brand in the input DataFrame (train/val/test).
def model_fix_wo_nan(pure_brand_model_df, input_df):
    # (brand, model_lower) -> model_correct for rows that need correction
    fix_map = (pure_brand_model_df.query("correction == 1")
               .set_index(['brand','model_lower'])['model_correct'])
    
    # replace model only where a correction exists; otherwise keep original
    keys = list(zip(input_df['Brand'], input_df['model']))
    input_df['model'] = pd.Series(keys, index=input_df.index).map(fix_map).fillna(input_df['model'])
    return input_df['model']

In [32]:
deduped_model_df = pure_brand_model_df.copy()

In [33]:
# List all rows where the model name is "i3" across brands
# BMW "i3" is correct; Hyundai "i3" is a typo (should be "i30").
deduped_model_df[deduped_model_df['model_lower'].duplicated(keep=False)]

,brand,model_lower,model_correct,correction
41,bmw,i3,i3,0
99,hyundai,i3,i30,1


In [34]:
# Drop Hyundai i3 rows by index from a query
deduped_model_df = deduped_model_df.drop(deduped_model_df.query("brand == 'hyundai' and model_lower == 'i3'").index)
deduped_model_df[deduped_model_df['model_lower'].duplicated(keep=False)]

,brand,model_lower,model_correct,correction


In [35]:
# Fix model names only for rows with unknown (NaN) Brand in the input DataFrame (train/val/test).
def model_fix_with_nan(deduped_model_df, input_df):
    fix_map = deduped_model_df.loc[deduped_model_df['correction'] == 1].set_index('model_lower')['model_correct']
    
    mask = input_df['Brand'].isna()
    input_df.loc[mask, 'model'] = input_df.loc[mask, 'model'].map(fix_map).fillna(input_df.loc[mask, 'model'])
    return input_df['model']

In [36]:
# Fill missing Brand values
def brand_fix_with_nan(deduped_model_df, input_df):
    # Rows where Brand is NaN in input_df
    input_df.loc[input_df["Brand"].isna()][["Brand","model"]]
    
    # Map input_df['model'] to short_brand_model_df['model_lower']
    # to fill missing Brand values in input_df with the correct brand names from short_brand_model_df.
    input_df['Brand'] = input_df['Brand'].replace('', pd.NA)  # treat empty as missing
    input_df['Brand'] = input_df['Brand'].fillna(input_df['model'].map(deduped_model_df.set_index('model_lower')['brand']))
    return input_df['Brand']

### **Fix Brand, model, transmission & fuelType**

In [ ]:
def fix_brand_model_transmission_fuelType(input_df, pure_brand_model_df, deduped_model_df):
    
    input_df['Brand'] = input_df['Brand'].str.lower().str.strip()
    input_df['Brand'] = input_df['Brand'].replace(BRAND_CORRECTIONS)

    input_df['transmission'] = input_df['transmission'].str.lower().str.strip()
    input_df['transmission'] = input_df['transmission'].replace(Transmission_Corrections)

    input_df['fuelType'] = input_df['fuelType'].str.lower().str.strip()
    input_df['fuelType'] = input_df['fuelType'].replace(fuelType_Corrections)

    # change all values to lower case and strip starting and ending spaces
    input_df['model'] = input_df['model'].str.lower().str.strip()
    
    input_df['model'] = model_fix_wo_nan(pure_brand_model_df, input_df)
    input_df['model'] = model_fix_with_nan(deduped_model_df, input_df)
    input_df['Brand'] = brand_fix_with_nan(deduped_model_df, input_df)

    input_df = input_df.dropna(subset=['Brand', 'model'])

    # Keep only rows whose model exists in the vocabulary
    input_df = input_df[input_df['model'].isin(pure_brand_model_df['model_lower'])]
    return input_df   

In [39]:
train = fix_brand_model_transmission_fuelType(train, pure_brand_model_df, deduped_model_df)

In [40]:
train.shape

(74260, 13)

## 2.2 Variable Exploration

### 2.2.1 Univariate Analysis

### 2.2.2 Bivariate Analysis

# 3. Data cleaning and Preprocessing

## 3.1 Duplicates

## 3.2 Feature Engeneering

### 3.2.1 Data Type Conversions

In [41]:
# Make sure the variable types are correctly identified

# Identifier
id_vars = ["carID"]
# Categorical variables (nominal, not ordered)
cat_vars = [
    "Brand",
    "model",
    "transmission",
    "fuelType"
]
# Ordinal variables (ordered but discrete)
ord_vars = [
    "year",
    "previousOwners"
]
# Continuous numerical variables
cont_vars = [
    "mileage",
    "tax",
    "mpg",
    "engineSize",
    "paintQuality%"
]
# Boolean variables
bool_vars = [
    "hasDamage"
]
# Variable groups in a dictionary
var_types = {
    "identifier": id_vars,
    "categorical": cat_vars,
    "ordinal": ord_vars,
    "continuous": cont_vars,
    "boolean": bool_vars
}
# Display 
for k, v in var_types.items():
    print(f"{k.capitalize():<12}: {v}")

Identifier  : ['carID']
Categorical : ['Brand', 'model', 'transmission', 'fuelType']
Ordinal     : ['year', 'previousOwners']
Continuous  : ['mileage', 'tax', 'mpg', 'engineSize', 'paintQuality%']
Boolean     : ['hasDamage']


In [47]:
# convert the variable types
for col in var_types['categorical']:
    train[col] = train[col].astype('category')

for col in var_types['ordinal']:
    train[col] = train[col].astype('category')

for col in var_types['continuous']:
    train[col] = train[col].astype('float')

for col in var_types['boolean']:
    train[col] = train[col].astype('bool')

In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74260 entries, 0 to 75972
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Brand           74260 non-null  category
 1   model           74260 non-null  category
 2   year            72804 non-null  category
 3   price           74260 non-null  int64   
 4   transmission    72766 non-null  category
 5   mileage         72834 non-null  float64 
 6   fuelType        72790 non-null  category
 7   tax             66517 non-null  float64 
 8   mpg             66494 non-null  float64 
 9   engineSize      72775 non-null  float64 
 10  paintQuality%   72762 non-null  float64 
 11  previousOwners  72743 non-null  category
 12  hasDamage       74260 non-null  bool    
dtypes: bool(1), category(6), float64(5), int64(1)
memory usage: 4.5 MB


### 3.2.2 Encoding